# Tvorba datasetu

### Obsah<a class='anchor' id='top'></a>
* [Globálne premenné](#globalne-premenne)
* [Pripojenie na Prometheus](#pripojenie-na-prometheus)
* [Časové vymedzenie](#casove-vymedzenie)
* [Načítanie a spracovanie metrík z Prometheus](#nacitanie-a-spracovanie-metrik-z-prometheus)
* [Spracovanie a extrakcia logov z funkcií Open5gs](#spracovanie-a-extrakcia-logov-z-funkcii-open5gs)
* [Agregácia a transformácia dát](#agregacia-a-transformacia-dat)
* [Triedenie logov](#triedenie-logov)
* [Spojenie metrík a logov](#spojenie-metrik-a-logov)
* [Mapovanie kategorických dát](#mapovanie-kategorickych-dat)
* [Vloženie informácie o akutálnom UC](#vlozenie-informacie-o-akutalnom-uc)
* [Uloženie datasetu](#ulozenie-datasetu)

In [1]:
import os
import re
from datetime import datetime, timedelta
from pathlib import Path

import pandas as pd
import pytz
from prometheus_api_client import PrometheusConnect

/Users/davidtruhlar/Documents/FIIT/BP/5GDigitalTwin/Implementation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
metrics = {
    "fivegs_smf": [
        "fivegs_smffunction_sm_n4sessionreportsucc",
        "fivegs_smffunction_sm_qos_flow_nbr",
    ],
    "fivegs_pcf": [
        "fivegs_pcffunction_pa_policysmassosucc",
        "fivegs_pcffunction_pa_sessionnbr",
    ],
    "fivegs_upf": [
        "fivegs_upffunction_sm_n4sessionreport",
    ],
    "fivegs_amf": [
        "fivegs_amffunction_mm_confupdate",
    ],
}

In [3]:
# Global variables
PROMETHEUS_PORT = 9090                                                                            # Port for Prometheus metrics
STEP = "1s"                                                                                       # Time step for the simulation
TIMEDELTA_SECONDS = 10                                                                            # Time delta for the simulation
LOCAL_TZ = pytz.timezone("Europe/Bratislava")                                                     # Timezone for the simulation
LOG_DIR = "/open5gs/install/var/log/open5gs"                                                      # Directory with logs
LOG_PATTERN = re.compile(r"(\d{2}/\d{2} \d{2}:\d{2}:\d{2}\.\d{3}):\s+\[(\w+)\]\s+(\w+):\s*(.+)")  # Regex pattern for log lines
ENCODING = "utf-8"                                                                                # Encoding for log files

<div class="alert alert-info">
<b>Príklad logu:</b> 04/02 11:05:03.836: [amf] INFO: [Added] Number of gNB-UEs is now 3 (../src/amf/context.c:2678)
</div>

In [4]:
# Connect to Prometheus
try:
    prom = PrometheusConnect(url=f"http://metrics:{PROMETHEUS_PORT}", disable_ssl=True)
except Exception as e:
    print(f"Error connecting to Prometheus: {e}")

<div class="alert alert-warning">
<b>Prometheus:</b> Pre správne fungovanie je potrebné sa uistiť, že Docker container 'metrics' je spustený a beží na porte 9090.<br>
60f374bfb155   docker_metrics           "/bin/bash -c /mnt/m…"   2 days ago     Up 30 minutes   0.0.0.0:9090->9090/tcp   
</div>

In [5]:
# Time range for the simulation
end_time = datetime.now()
start_time = end_time - timedelta(seconds=TIMEDELTA_SECONDS)

In [ ]:
# # Input range (UTC)
# start_str = "2025-04-14 12:52:30"
# end_str = "2025-04-14 12:56:30"

# # Parse as UTC datetime
# start_utc = datetime.fromisoformat(start_str)
# end_utc = datetime.fromisoformat(end_str)

# # Convert to local time with microseconds
# start_local = start_utc.astimezone().replace(microsecond=0)
# end_local = end_utc.astimezone().replace(microsecond=0)

# print("Start time:", start_utc)
# print("End time:", end_utc)

# end_time = end_utc
# start_time = start_utc

<div class="alert alert-info">
<b>Zakomentovaný kód:</b> Bunka vyššie slúžila na špecifikovanie časového rozsahu pre zber dát. Keď bol vytváraný reálny dataset, čas bol zaznamenaný, aby sme mohli neskôr načítať dáta zo servera Prometheus.  
</div>

### Načítanie a spracovanie metrík z Promethus

In [6]:
# Create an empty list to hold metric DataFrames
df_list = []

# Iterate through metric groups
for group, metric_list in metrics.items():
    for metric in metric_list:
        try:
            response = prom.custom_query_range(
                metric, start_time=start_time, end_time=end_time, step=STEP
            )

            if not response:
                print(f"⚠️ No data: {group}/{metric}")
                continue

            for entry in response:
                base_metric_name = entry["metric"].get("__name__", metric)

                if "values" in entry and isinstance(entry["values"], list):
                    extracted_values = [
                        (
                            datetime.utcfromtimestamp(int(ts))
                            .replace(tzinfo=pytz.utc)
                            .astimezone(LOCAL_TZ),
                            float(val)
                        )
                        for ts, val in entry["values"]
                    ]

                    metric_df = pd.DataFrame(extracted_values, columns=["timestamp", "value"])
                    metric_df["metric_name"] = base_metric_name
                    metric_df["group"] = group

                    df_list.append(metric_df)
                else:
                    print(f"⚠️ No valid values found: {group}/{metric}")

        except Exception as e:
            print(f"❌ Error fetching {group}/{metric}: {e}")

# Combine all metrics into one DataFrame
if df_list:
    final_df = pd.concat(df_list, ignore_index=True)
    final_df['timestamp'] = final_df['timestamp'].astype(str).str.replace(r'\+\d{2}:\d{2}', '', regex=True)
    final_df["timestamp"] = pd.to_datetime(final_df["timestamp"])
else:
    print("❌ No data collected for any metric.")

❌ Error fetching fivegs_smf/fivegs_smffunction_sm_n4sessionreportsucc: HTTPConnectionPool(host='metrics', port=9090): Max retries exceeded with url: /api/v1/query_range?query=fivegs_smffunction_sm_n4sessionreportsucc&start=1746437684&end=1746437694&step=1s (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x109ff9040>: Failed to resolve 'metrics' ([Errno 8] nodename nor servname provided, or not known)"))
❌ Error fetching fivegs_smf/fivegs_smffunction_sm_qos_flow_nbr: HTTPConnectionPool(host='metrics', port=9090): Max retries exceeded with url: /api/v1/query_range?query=fivegs_smffunction_sm_qos_flow_nbr&start=1746437684&end=1746437694&step=1s (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x109ff96a0>: Failed to resolve 'metrics' ([Errno 8] nodename nor servname provided, or not known)"))
❌ Error fetching fivegs_pcf/fivegs_pcffunction_pa_policysmassosucc: HTTPConnectionPool(host='metrics', port=9090): Max retries exceeded with u

KeyboardInterrupt: 

<div class="alert alert-info">
<b>final_df:</b> je dataframe, ktorý obsahuje všetky potrebné metriky z Prometheus.<br>
</div>

#### Spracovanie a extrakcia logov z funkcií Open5gs

In [30]:
log_data = []

# Aplikácie, ktoré chceme sledovať
allowed_applications = {"amf", "upf", "smf", "udm", "gmm"}

for log_path in Path(LOG_DIR).glob("*.log"):
    try:
        with open(log_path, "r", encoding=ENCODING, errors="ignore") as f:
            for line in f:
                match = LOG_PATTERN.match(line)
                if match:
                    timestamp_str, application, log_level, log_message = match.groups()

                    if application.lower() not in allowed_applications:
                        continue

                    # Konvertujeme timestamp na datetime
                    log_timestamp = datetime.strptime(timestamp_str, "%m/%d %H:%M:%S.%f")
                    log_timestamp = log_timestamp.replace(year=start_time.year, microsecond=0)

                    # Nechávame len logy v časovom intervale
                    if log_timestamp > start_time and log_timestamp < end_time:
                        log_data.append({
                            "timestamp": log_timestamp,
                            "application": application,
                            "log_level": log_level,
                            "log_message": log_message
                        })
                    else:
                        continue
    except Exception as e:
        print(f"❌ Failed to process {log_path.name}: {e}")

log_data = pd.DataFrame(log_data)

<div class="alert alert-info">
<b>log_data:</b> je dataframe, ktorý obsahuje všetky potrebné logy z Open5gs.<br>

#### Agregácia a transformácia údajov o metrikách

In [ ]:
# Data aggregation
data_agg = final_df.groupby(["timestamp", "metric_name"])["value"].mean().reset_index()
data_pivot = data_agg.pivot(index="timestamp", columns="metric_name", values="value")

# Adding "_value" to column names
data_pivot.columns = [f"{col}_value" for col in data_pivot.columns]

# Adding a time column
data_pivot.reset_index(inplace=True)

In [ ]:
# Logs classification
patterns = {
    "remove": re.compile(r"\b(Removed|Deregister|De-register|Implicit De-registered)\b", re.IGNORECASE),
    "refused": re.compile(r"\b(refused|connection refused)\b", re.IGNORECASE),
    "number_of_sessions_or_ues": re.compile(r"\b(Number of (gNBs|AMF-UEs|AMF-Sessions|gNB-UEs))\b", re.IGNORECASE),
    "registration": re.compile(r"\b(Registration request|InitialUEMessage|Added|Unknown UE by SUCI|SUCI)\b", re.IGNORECASE),
    "error": re.compile(r"\b(ERROR)\b", re.IGNORECASE),
    "warning": re.compile(r"\b(WARNING)\b", re.IGNORECASE),
}

def classify_log_message(message):
    if not isinstance(message, str):
        return "nothing"
    for label, pattern in patterns.items():
        if pattern.search(message):
            return label
    return "nothing"

# Before adding the "log_type" column, check if the "log_message" column exists
if "log_message" in log_data.columns:
	log_data["log_type"] = log_data["log_message"].apply(classify_log_message)

In [ ]:
# Shorten the DataFrame to only include relevant columns
logs_short = log_data[["timestamp", "application", "log_type"]] \
	if not log_data.empty \
	else pd.DataFrame(columns=["timestamp", "application", "log_type"])

In [ ]:
# Logs are in UTC, so we need to convert them to the local timezone
logs_short["timestamp"] = pd.to_datetime(logs_short["timestamp"], errors='coerce').dt.tz_localize("UTC")

# Logs filtering
logs_short = logs_short[logs_short["timestamp"] >= start_time.replace(tzinfo=pytz.utc)]

# Remove timezone from timestamp to match the format in Prometheus data
logs_short["timestamp"] = logs_short["timestamp"].dt.tz_localize(None)

In [ ]:
# Drop duplicates
logs_short = logs_short.drop_duplicates(subset=["timestamp", "application", "log_type"])
logs_short = logs_short.sort_values(by=["timestamp", "application", "log_type"])
logs_short = logs_short.reset_index(drop=True)

<div class="alert alert-info">
<b>logs_short:</b> tento dataframe budeme spájať s dataframe-om final_df.
</div>

In [ ]:
# Metrics and logs merging
merged_data = pd.merge(data_pivot, logs_short, on="timestamp", how="outer")

In [ ]:
featured_metrics = {"features": ["fivegs_pcffunction_pa_sessionnbr_value", "fivegs_upffunction_sm_n4sessionreport_value", "fivegs_amffunction_mm_confupdate_value", "fivegs_pcffunction_pa_policysmassosucc_value", "fivegs_smffunction_sm_qos_flow_nbr_value", "fivegs_smffunction_sm_pdusessioncreationsucc_value", "log_type", "application"]}
featured_metrics = featured_metrics["features"]

missing_metrics = []
for metric in featured_metrics:
    if metric not in merged_data.columns:
        missing_metrics.append(metric)
if missing_metrics:
    print("❌ Missing metrics in final DataFrame:")
    for metric in missing_metrics:
        print(f"  - {metric}")

# Add columns for missing metrics
for metric in missing_metrics:
    merged_data[metric] = 0.0


# Order the columns to match the selected features
ordered_columns = ["timestamp"] + featured_metrics
for col in merged_data.columns:
    if col not in ordered_columns:
        ordered_columns.append(col)
merged_data = merged_data[ordered_columns]

❌ Missing metrics in final DataFrame:
  - fivegs_pcffunction_pa_sessionnbr_value
  - fivegs_pcffunction_pa_policysmassosucc_value
  - fivegs_smffunction_sm_pdusessioncreationreq_value
  - fivegs_smffunction_sm_qos_flow_nbr_value


In [ ]:
# Based on predefined mappings we convert the application and log_type columns to numerical values
APP_MAP = {"0": 0, "amf": 1, "gmm": 2, "udm": 3, "smf": 4, "upf": 5}
LOG_MAP = {"0": 0, "registration": 1, "number_of_sessions_or_ues": 2, "nothing": 3, "remove": 4, "error": 5}

merged_data['application'] = merged_data['application'].map(APP_MAP).fillna(0)
merged_data['log_type'] = merged_data['log_type'].map(LOG_MAP).fillna(0)

In [ ]:
current_uc = None

# Load the current UC from a file
with open("./data/current_uc.txt", "r") as f:
    current_uc = f.read().strip()

# If the current UC is valid, add it to the DataFrame
if not merged_data.empty:
    merged_data["current_uc"] = current_uc

UC_MAP = {"uc1": 0, "uc2": 1, "uc3": 2, "uc4": 3, "uc5": 4, "uc6": 5}

# Convert the current_uc column to numerical values
merged_data['current_uc'] = merged_data['current_uc'].map(UC_MAP).fillna(0)

<div class="alert alert-warning">
<b> Mapovanie:</b> je potrebné zabezpečiť aby bol súbor 'current_uc.txt' vytvorený a v správnom adresári.

In [ ]:
csv_file = "./data/running_data.csv"

# If the file doesn't exist or is empty, we write the header
write_header = not os.path.exists(csv_file) or os.path.getsize(csv_file) == 0

# Filter only new records if file exists and is not empty
if not write_header:
    try:
        last_timestamp = pd.read_csv(csv_file, usecols=["timestamp"])["timestamp"].max()
        merged_data = merged_data[merged_data["timestamp"] > last_timestamp]
    except Exception as e:
        print(f"⚠️ Issue reading existing CSV: {e}. Proceeding without filtering.")

# Add new data to the CSV file
if not merged_data.empty:
    merged_data.to_csv(csv_file, mode="a", index=False, header=write_header)

<div class="alert alert-success">
<b>Uloženie datasetu:</b> je potrebné zabezpečiť aby bol súbor 'running_data.csv' vytvorený a v správnom adresári.
</div>